In [6]:
import attr
import time
import pprint
from schematics.models import Model
from schematics.types import StringType, IntType
from marshmallow import Schema, fields


@attr.s
class PersonAttrs:
    name = attr.ib(validator=[
        attr.validators.instance_of(str)
    ])
    age = attr.ib(validator=[
        attr.validators.instance_of(int)
    ])


class PersonSchematics(Model):
    name = StringType()
    age = IntType()


class PersonMarshmallow(Schema):
    name = fields.Str()
    age = fields.Int()

marshmallow_schema = PersonMarshmallow()

target = {"name": "Mario Luigi", "age": 32}

TRIALS = 10000


def benchmark(name, func):
    start_time = time.time()
    for i in range(TRIALS):
        func()
    elapsed_time = time.time() - start_time
    return [name, elapsed_time, TRIALS]

data = []
data.append(benchmark("schematics", lambda: PersonSchematics(target).validate()))
data.append(benchmark("marshmallow", lambda: marshmallow_schema.dump(target)))
data.append(benchmark("attrs", lambda: PersonAttrs(**target)))

pprint.pprint(data)

[['schematics', 1.3649110794067383, 10000],
 ['marshmallow', 0.23163890838623047, 10000],
 ['attrs', 0.024604082107543945, 10000]]
